In [1]:
# 네이버 검색 API 예제 - 블로그 검색
import pandas as pd
import requests
import os
import sys
import urllib.request
import json
import datetime
import xlsxwriter


In [53]:
def get_search_order():
    keyword = input("키워드 : ")
      
    return keyword

def get_client_info():
    client = pd.read_csv('~/API/naver_API.csv')
    client_id = client.loc[0,'client_id']
    client_secret = client.loc[0,'client_secret']

    return client_id,client_secret

def gen_search_url(node, search_text, start_num, disp_num):
    base = "https://openapi.naver.com/v1/search"
    node = "/" + node + ".json"
    param_query = "?query=" + urllib.parse.quote(search_text)
    param_start = "&start=" + str(start_num)
    param_disp = "&display=" + str(disp_num)
    
    return base + node + param_query + param_start + param_disp

def get_result_onpage(client_id,client_secret,url):
    request = urllib.request.Request(url)
    request.add_header("X-Naver-Client-Id",client_id)
    request.add_header("X-Naver-Client-Secret",client_secret)
    response = urllib.request.urlopen(request)
    print("[%s] Url Requests Success" % datetime.datetime.now())

    return json.loads(response.read().decode("utf-8"))

def get_fields(json_data):
    title = [delete_tag(i["title"]) for i in json_data["items"]]
    description = [delete_tag(i["description"]) for i in json_data["items"]]
    postdate = [i["postdate"] for i in json_data["items"]]
    bloggername = [i["bloggername"] for i in json_data["items"]]
    df = pd.DataFrame({
        "title" : title,
        "description" : description,
        "bloggername" : bloggername,
        "postdate" : postdate,
        })

    return df

def delete_tag(input_str):
    input_str = input_str.replace("<b>","")
    input_str = input_str.replace("</b>","")
    return input_str

def actMain(kwd):
    total_result = []
    client_id,client_secret = get_client_info()
    for i in range(1,1000,100):
        url = gen_search_url("blog",kwd,i,100)
        one_result = get_result_onpage(client_id,client_secret,url)
        df = get_fields(one_result)      
        total_result.append(df)
    
    total_result = pd.concat(total_result)
    total_result.reset_index(drop=True, inplace=True)
    return total_result

def export_file(keyword,total_result):
    writer = pd.ExcelWriter(f"/Users/young/git/project/export/{keyword}_{datetime.datetime.now()}.xlsx", engine="xlsxwriter")
    total_result.to_excel(writer,sheet_name='Sheet1')
    workbook = writer.book
    worksheet = writer.sheets["Sheet1"]
    writer.save()
    pass

def init():
    keyword = get_search_order()
    total_result = actMain(keyword)
    export_file(keyword,total_result)


init()

[2023-01-18 22:42:25.490614] Url Requests Success
[2023-01-18 22:42:25.810113] Url Requests Success
[2023-01-18 22:42:26.164760] Url Requests Success
[2023-01-18 22:42:26.679566] Url Requests Success
[2023-01-18 22:42:27.287161] Url Requests Success
[2023-01-18 22:42:27.764725] Url Requests Success
[2023-01-18 22:42:28.289989] Url Requests Success
[2023-01-18 22:42:28.932295] Url Requests Success
[2023-01-18 22:42:29.547452] Url Requests Success
[2023-01-18 22:42:30.264107] Url Requests Success
